# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Loan Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/complaints.csv",
    metadata_columns=[
      "Date received", 
      "Product", 
      "Sub-product", 
      "Issue", 
      "Sub-issue", 
      "Consumer complaint narrative", 
      "Company public response", 
      "Company", 
      "State", 
      "ZIP code", 
      "Tags", 
      "Consumer consent provided?", 
      "Submitted via", 
      "Date sent to company", 
      "Company response to consumer", 
      "Timely response?", 
      "Consumer disputed?", 
      "Complaint ID"
    ]
)

loan_complaint_data = loader.load()

for doc in loan_complaint_data:
    doc.page_content = doc.metadata["Consumer complaint narrative"]

Let's look at an example document to see if everything worked as expected!

In [4]:
loan_complaint_data[0]

Document(metadata={'source': './data/complaints.csv', 'row': 0, 'Date received': '03/27/25', 'Product': 'Student loan', 'Sub-product': 'Federal student loan servicing', 'Issue': 'Dealing with your lender or servicer', 'Sub-issue': 'Trouble with how payments are being handled', 'Consumer complaint narrative': "The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers.", 'Company public response': 'None', 'Company'

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "LoanComplaints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    loan_complaint_data,
    embeddings,
    location=":memory:",
    collection_name="LoanComplaints"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the complaints in the provided context, appears to be problems related to mismanagement and inaccurate handling of student loans. Specific issues include errors in loan balances, misapplied payments, wrongful denials of payment plans, incorrect reporting of account status (such as falsely being marked 90 days delinquent), improper transfer of loans without proper notification, and difficulty applying payments correctly—often resulting in higher interest accumulation and credit reporting discrepancies. These problems indicate that a prevalent issue is the mishandling and administrative errors in loan servicing.'

In [11]:
naive_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided data, some complaints indicate that they were not handled in a timely manner. Specifically, the complaint with Complaint ID 12709087, submitted to MOHELA on 03/28/25, was not responded to promptly, as it was marked "Timely response?": "No." The consumer reports that they have not heard from the company despite multiple attempts, and the issue remains unresolved over a period that includes multiple follow-ups.\n\nAdditionally, the complaint with Complaint ID 13062402, submitted to Nelnet on 04/18/25, was addressed within the timeframe (marked "Timely response?": "Yes"), so this particular complaint was handled timely.\n\nIn summary, yes, at least one complaint (ID 12709087) was not handled in a timely manner.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"People failed to pay back their loans primarily due to a combination of factors including:\n\n1. **Accumulation of interest during forbearance or deferment periods**: Many borrowers reported that while they were granted options like forbearance or deferment, the interest continued to accrue, making the total debt larger over time and extending the repayment period.\n\n2. **Inadequate or confusing communication from loan servicers**: Several complaints indicated that borrowers were not properly notified about loan transferences, repayment start dates, or changes in their loan status. This lack of information led to missed payments or unawareness of repayment obligations.\n\n3. **Financial hardships and stagnant wages**: Borrowers experienced difficulty in making payments because their income did not increase commensurately with their debts, especially after taking on loans early in life under the assumption of higher future earnings.\n\n4. **Failed or unavailable repayment options**: S

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(loan_complaint_data, )

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the provided complaints, appears to be issues related to dealing with lenders or servicers. Specifically, concerns such as incorrect or bad information about the loan, trouble with how payments are being handled (including applying payments or paying more in interest than principal), and disputes over fees or reimbursements are prominent. These issues reflect difficulties borrowers face in managing or understanding their loans, often involving miscommunication, errors, or perceived unfair practices by loan servicers.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, all the complaints documented indicate that responses from the companies were marked as "Closed with explanation" and the response times are noted as "Yes" to being timely. Therefore, it appears that no complaints were left unresolved or handled outside the appropriate timeframe.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including issues related to mishandling or miscommunication by loan servicers, such as being steered into incorrect types of forbearances, unenrolled from autopay without notification, or experiencing repeated payment reversals due to errors with their bank or loan servicing company. In some cases, borrowers were unaware of transfers to new loan servicers or did not receive proper notices, leading to missed payments and negative credit impacts. Additionally, some borrowers faced obstacles like unresponsive or unhelpful servicers, which prevented them from resolving payment issues or accessing help with COVID-related or financial hardship deferments.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

#### ✅ ANSWER

BM25 is best at answering specific / direct questions. It does so by simply finding the document with key words that match the query at a high frequency.

Example: “Ubuntu 20.04 install NVIDIA driver 470”
Why BM25 wins:
•	Technical queries with version numbers or specific commands match better with exact tokens. There is likely a small number of documents whose keywords match the above and BM25 will prioritize these.
•	Embedding models often treat numbers and commands less precisely. It may retrieve documents for generally how to install different drivers on Ubuntu and certainly not for the specified version.

For factual, high-precision queries, BM25 is often the more reliable choice.

Other examples:
- "Python list comprehension syntax"
- "Section 230 of the Communications Decency Act"
- "ISO 27001 compliance checklist PDF"
- "New York Times January 5 2020 front page headline"

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issue with loans appears to be problems related to dealing with lenders or servicers, including errors, miscommunication, and mishandling of loan information. Specific issues frequently mentioned include errors in loan balances, misapplied payments, wrongful denials of payment plans, and inaccurate or incomplete information provided to borrowers.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, there are indications that some issues have not been handled in a timely manner. For example, one complaint mentions that it has been nearly 18 months with no resolution and the complainant is still awaiting a response and resolution despite over a year having passed since the initial request. Another complaint notes a problem that has persisted for 2-3 weeks without resolution. Additionally, some complaints also explicitly mention delays or unresolved issues over extended periods, such as over one year.\n\nTherefore, yes, some complaints did not get handled in a timely manner.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans mainly due to lack of awareness and understanding about their loan obligations, issues with loan management, and financial hardships. Specifically, some borrowers were not informed that they would need to repay their loans, leading to unawareness of the repayment requirement. Others faced difficulties with the handling and transfer of their loans between servicers, which caused confusion and disrupted communication. Additionally, many borrowers experienced financial hardships because of accruing interest during forbearance or deferment, making it difficult to pay down the principal. Some found the available repayment options, such as forbearance or income-based plans, resulted in increased interest accumulation, extending repayment periods and increasing total debt. Overall, poor communication, lack of proper information, and economic challenges contributed to the failure to repay loans.\n'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

"The most common issue with loans, based on the complaints provided, appears to be problems related to how loans are managed and serviced. This includes issues such as:\n\n- Trouble with payments being handled or applied correctly, often with additional funds being applied mostly to interest rather than principal.\n- Lack of clear communication or notification about loan status, payments, or changes.\n- Incorrect or fraudulent reporting that damages credit scores.\n- Errors in loan balances, misapplied payments, and wrongful denials of payment plans.\n- Problems with loan transfer, sale, or administration changes without proper notice.\n- Issues with interest rate increases, improper fees, or unauthorized changes to loan terms.\n- Mishandling of forbearance and repayment options, leading to increased balances and difficulty in repayment.\n- Breach of borrower rights and privacy violations.\n\nOverall, the most common issue tends to revolve around **servicing errors, miscommunication, a

In [26]:
multi_query_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints data, yes, some complaints indicate that issues were not handled in a timely manner. For example:\n\n- Complaint ID 12709087 (submitted 04/01/25) regarding unprocessed graduate loan application: the response was "No" for timely response, and the issue had been ongoing for over a year without resolution.\n- Complaint ID 12823876 (submitted 04/04/25) involved unrecorded payments, with the complaint indicating attempts to resolve for nearly two and a half years without a resolution, despite persistent follow-up.\n- Complaint ID 12739706 (submitted 04/01/25) about failure to inform about delinquent loans was also marked "No" for timely response.\n- Several other complaints note delays of months or multiple years in resolving issues or receiving responses.\n\nIn summary, multiple complaints reflect that certain complaints did not get addressed promptly or within a reasonable time frame.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans mainly due to issues such as mismanagement of their student loans, lack of proper communication from loan servicers, incorrect or delayed reporting of delinquency or default, and not being informed of or provided access to alternative repayment options like income-driven repayment plans or loan rehabilitation. Additionally, systemic problems such as errors in loan balances, interest capitalization, and transfers of loans without proper notice contributed to difficulties in repayment. Some borrowers also experienced hardships like financial distress, unemployment, or health issues, which made timely repayment impossible, especially when servicers did not offer support or guidance to help them manage their loans effectively.'

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

### ✅ ANSWER
Often there are many ways to portray the same piece of information for example:
- The Capital of France is Paris
- The main political center of France is Paris
- The epicenter of France is Paris

Recall measures the likelyhood that a relevant document is retrieved. 
By formulating different versions of the same question, you "target" the vector store from different "angles" increasing the chance that you hit and recall a relevant document no matter how that key piece of information is expressed. 

Also different user may ask the same question in different ways. Through question generation we may be able to reformulate the question in the way that that information is expressed in the vector store.

In class we also talked about pre-pending additional explanatory information to a document to improve recall and relevance. If a document is retrieved from a reformulated question it's possible to add details to the chunk that makes it more relateable to the original query.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = loan_complaint_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [31]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the provided complaints, appears to be problems related to federal student loan servicing, including errors in loan balances, misapplied payments, wrongful denials of payment plans, and misconduct by loan servicers. Other frequent issues involve discrepancies in credit reporting, unfair increases in interest rates, and poor communication or handling by lenders or servicers.'

In [34]:
parent_document_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Yes, according to the provided information, several complaints did not get handled in a timely manner. Specifically, complaints related to student loans serviced by MOHELA on 03/28/25 and 04/11/25 were marked as "Timely response?": "No," indicating they were not handled promptly. Additionally, the complaint regarding the dispute settlement sent to credit bureaus by Nelnet on 04/27/25 also had no response after over 30 days, which suggests a delay in handling that complaint.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans primarily due to a variety of issues, including:\n\n1. Lack of clear communication and proper notification from loan servicers about payment obligations, due dates, or changes in loan management.\n2. Difficulties caused by the closure or financial instability of educational institutions, leading to misrepresentation about the value of degrees and employment prospects, making it harder for borrowers to find jobs and generate income to repay loans.\n3. Severe financial hardship or personal issues that made consistent repayment unfeasible, often compounded by increased interest due to deferments and forbearances.\n4. Problems related to loan processing, such as inaccurate reporting, inability to verify debt legitimacy, and administrative errors, which further hindered repayment efforts.\n5. Borrowers also reported experiencing health issues and other personal circumstances that negatively impacted their ability to meet repayment obligations.\n\nIn su

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [36]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided data, the most common issue with loans appears to be related to dealing with lenders or servicers, particularly involving bad information about loans, mismanagement, incorrect account statuses, and poor communication. Several complaints highlight errors in loan balances, misapplied payments, wrongful collection or default notices, and difficulties accessing accurate loan information. \n\nWhile other issues such as problematic payment handling, improper reporting, and violations of privacy laws are also frequent, the predominant theme is dissatisfaction with how lenders or servicers manage and communicate about student loans.'

In [39]:
ensemble_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Yes, according to the provided complaints, several complaints were not handled in a timely manner. Specifically, at least two complaints indicated that responses were delayed beyond the expected timeframes:\n\n- One complaint (row 66) about incorrect account status reported by Nelnet was marked as "Timely response?": "No".\n- Another complaint (row 70) regarding incorrect repayment start date also was marked as "Timely response?": "No".\n- Similarly, a complaint (row 128) about delinquent loans reported by Mohela was also marked as "Timely response?": "No".\n- Additional complaints about unresolved issues and lack of response also highlight delays or lack of proper handling.\n\nWhile some complaints received responses, multiple cases show delays longer than standard response times, indicating that some complaints did not get handled in a timely manner.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for several reasons, including:\n\n1. Lack of clear information about repayment options and interest accumulation, leading borrowers to become overwhelmed by interest and long-term debt.\n2. Limited or misleading communication from loan servicers, such as not notifying borrowers of payment due dates, loan transfers, or changes in account status.\n3. Difficulty qualifying for or being informed about income-driven repayment plans or loan forgiveness programs.\n4. Economic hardships like unemployment, low income, or unexpected life events (e.g., homelessness, accidents, financial crises) making repayment unaffordable.\n5. Administrative errors, such as incorrect reporting of loan status, errors in payment application, or mismanagement of loan accounts.\n6. Debt being continuously accumulated due to unaddressed interest capitalization, forbearance steering practices, or other predatory lending behaviors.\n7. Lack of awareness or understanding about lo

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [42]:
semantic_documents = semantic_chunker.split_documents(loan_complaint_data[:20])

Let's create a new vector store.

In [43]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Loan_Complaint_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issues with loans appear to involve difficulties in repayment, including problems with loan forgiveness or discharge, incorrect or improper reporting, trouble with payment plans, and issues with accurate or timely information about the loan status or balances. Many complaints also relate to lack of transparency, conflicting information from loan servicers, and delays or failures in processing requests or documentation.\n\nTherefore, the most common issues seem to be related to:\n\n- Struggles to repay or manage student loans\n- Errors or disputes regarding loan reporting and account status\n- Problems with loan forgiveness or discharge processes\n- Lack of clear communication or transparency from loan servicers\n\nIf you are asking for a specific "most common issue," based on the data, it appears that difficulties in repayment and communication issues with servicers are prevalent.'

In [47]:
semantic_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, it appears that all the complaints listed received responses from the companies, and those responses indicated that the issues were "Closed with explanation" and were marked as "Timely response." There is no indication that any complaint was not handled in a timely manner.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"Based on the provided context, people failed to pay back their loans for several reasons, including:\n\n1. Lack of clear or accurate information from loan servicers, leading to confusion about their loan status or repayment terms (e.g., receiving bad information about their loan or receiving conflicting instructions).\n2. Administrative or processing issues, such as missing payments or problems with payment processing, which hinder their ability to stay current.\n3. Deliberate delays or stalling tactics by loan servicers or their representatives, making it difficult for borrowers to clarify or resolve issues, leading some to give up.\n4. Disputes over the legitimacy of the debt or improper reporting, sometimes due to legal or privacy breaches, which may cause confusion or resistance to repayment.\n5. Changes in payment obligations or re-amortization delays after forbearance periods, causing unexpected increases in monthly payments, which may overwhelm borrowers.\n6. Being unaware of t

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?



# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [50]:
import os
import getpass

In [52]:
os.environ["LANGSMITH_PROJECT"] = "Advanced RAG"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API Key:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [57]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


docs = loan_complaint_data
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:5], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/3 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

Node 18d9330c-8e4a-400f-a6c3-02ed6cb935ac does not have a summary. Skipping filtering.
Node cd6f50a6-974f-4bb3-8ca9-733578116056 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/13 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [58]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,why my payment go up so much after federal stu...,[The federal student loan COVID-19 forbearance...,The federal student loan COVID-19 forbearance ...,single_hop_specifc_query_synthesizer
1,Why did Aidvantage assign me an incorrect mont...,[I submitted my annual Income-Driven Repayment...,Aidvantage assigned you a monthly payment amou...,single_hop_specifc_query_synthesizer
2,What does it mean if my personal and financial...,[My personal and financial data was compromise...,If your personal and financial data was compro...,single_hop_specifc_query_synthesizer
3,Why studentaid.gov say my issuer is nelnet but...,"[According to Studentaid.gov, Im to get an ema...","According to studentaid.gov, you are supposed ...",single_hop_specifc_query_synthesizer
4,wen do i gotta start payin my fedral loan paym...,[Since the resumption of federal loan payments...,"Since the resumption of federal loan payments,...",single_hop_specifc_query_synthesizer
5,"How does student loan servicing confusion, suc...","[<1-hop>\n\nAccording to Studentaid.gov, Im to...","Student loan servicing confusion, including co...",multi_hop_abstract_query_synthesizer
6,Why is it so hard to make payments after the e...,"[<1-hop>\n\nAccording to Studentaid.gov, Im to...",It is hard to make payments after the end of C...,multi_hop_abstract_query_synthesizer
7,Why is it so hard to make payments on my stude...,"[<1-hop>\n\nAccording to Studentaid.gov, Im to...",It’s hard to make payments on your student loa...,multi_hop_abstract_query_synthesizer
8,Why is it so hard to make payments on my stude...,"[<1-hop>\n\nAccording to Studentaid.gov, Im to...",It is difficult to make payments on your stude...,multi_hop_abstract_query_synthesizer
9,Why is it so hard to make payments when my ser...,"[<1-hop>\n\nAccording to Studentaid.gov, Im to...",It is hard to make payments because studentaid...,multi_hop_abstract_query_synthesizer


In [60]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data - Advanced Retrieval"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data - Advanced Retrieval"
)

In [61]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [108]:
naive_retrieval_chain.name = "naive_retrieval"
bm25_retrieval_chain.name = "bm25_retrieval"
contextual_compression_retrieval_chain.name = "contextual_compression_retrieval"
multi_query_retrieval_chain.name = "multi_query_retrieval"
parent_document_retriever.name = "parent_document_retrieval"
ensemble_retrieval_chain.name = "ensemble_retrieval"
semantic_retrieval_chain.name = "semantic_retrieval"

retrievers = [
    naive_retrieval_chain,
    bm25_retrieval_chain,
    contextual_compression_retrieval_chain,
    multi_query_retrieval_chain,
    parent_document_retriever,
    ensemble_retrieval_chain,
    semantic_retrieval_chain,
]



In [76]:
retrievers[0].name

'Naive Retrieval'

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [158]:
import uuid
from langsmith.evaluation import evaluate

def langsmith_evaluation(rag_chain, dataset_name):
    logger.info(f"Starting LangSmith evaluation for chain: {rag_chain.__class__.__name__}")
    logger.info(f"Using dataset: {dataset_name}")
    id = str(uuid.uuid4())
    
    # Define a wrapper function that handles the input format
    def chain_invoke(inputs):
        # Extract just the question string from the inputs
        if isinstance(inputs, dict) and "question" in inputs:
            query = inputs["question"]
        else:
            query = str(inputs)
            
        # For retrievers that are not chains
        if hasattr(rag_chain, "_get_relevant_documents"):
            return {"output": rag_chain._get_relevant_documents(query)}
        # For chain-based retrievers
        else:
            return rag_chain.invoke({"question": query})
    
    try:
        evaluate(
            chain_invoke,
            data=dataset_name,
            evaluators=[],
            experiment_prefix=rag_chain.name,
            metadata={"revision_id": f"{rag_chain.name}_{id}"},
        )
        logger.info("LangSmith evaluation completed successfully")
    except Exception as e:
        logger.error(f"Error during LangSmith evaluation: {str(e)}")
        raise

In [89]:
pd_dataset = dataset.to_pandas()

In [90]:
len(pd_dataset)

11

In [92]:
pd_dataset.iloc[2].user_input


'What does it mean if my personal and financial data was compromised in violation of FERPA?'

In [104]:
for i, test_row in enumerate(dataset):
  print(test_row.eval_sample.user_input)

why my payment go up so much after federal student loan COVID-19 forbearance program end?
Why did Aidvantage assign me an incorrect monthly payment amount on my Income-Driven Repayment plan despite my income qualifying for a lower payment, and what actions have they taken in response to my inquiries?
What does it mean if my personal and financial data was compromised in violation of FERPA?
Why studentaid.gov say my issuer is nelnet but nelnet say my issuer is somewhere else and now I dont know where my loan is and I only found out cause I tried to pay and now I might get more interest and late fees?
wen do i gotta start payin my fedral loan payments?
How does student loan servicing confusion, such as conflicting information about loan servicers and lack of notification about servicer changes, impact the process and accuracy of Income-Driven Repayment (IDR) recertification, as seen in issues with both Nelnet and Aidvantage?
Why is it so hard to make payments after the end of COVID-19 fo

In [156]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate as ragas_evaluate, RunConfig, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
import copy

custom_run_config = RunConfig(timeout=360)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

def ragas_evaluation(rag_chain, dataset):
  print(f"Starting RAGAS evaluation for chain: {rag_chain.__class__.__name__}")
  
  working_dataset = copy.deepcopy(dataset)
  print(f"Processing {len(working_dataset)} samples")

  for i, test_row in enumerate(working_dataset):
    print(f"Processing sample {i+1}/{len(working_dataset)}")
    print(f"eval_sample: {test_row.eval_sample}")
    print(f"type: {type(test_row.eval_sample)}")
    print(f"Question: {test_row.eval_sample.user_input}")
    print(f"Reference: {test_row.eval_sample.reference}")
    print(f"Reference Contexts: {test_row.eval_sample.reference_contexts}")
    response = rag_chain.invoke({"question" : test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"].content
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    print(f"Response: {test_row.eval_sample.response}")
    print(f"Retrieved Contexts ({type(test_row.eval_sample.retrieved_contexts)}): {test_row.eval_sample.retrieved_contexts}")
    print(f"Retrieved {len(test_row.eval_sample.retrieved_contexts)} contexts")

  print("Creating evaluation dataset")
  try:
    evaluation_dataset = EvaluationDataset.from_pandas(working_dataset.to_pandas())
  except Exception as e:
    print(f"Error creating evaluation dataset: {e}")
    return None

  print("Running RAGAS evaluation metrics")
  result = ragas_evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
  )
  
  print("RAGAS evaluation complete")
  print(f"Results: {result}")
  
  return result

In [145]:
metrics = {}
for chain in retrievers:
  langsmith_evaluation(chain, dataset_name)

INFO:__main__:Starting LangSmith evaluation for chain: RunnableSequence
INFO:__main__:Using dataset: Loan Synthetic Data - Advanced Retrieval


View the evaluation results for experiment: 'naive_retrieval-25675490' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/85bedb6e-6ef2-4c08-9a48-4984b66a036c/compare?selectedSessions=e60e78cc-e197-4596-9d98-3263dcd18d90




0it [00:00, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https

View the evaluation results for experiment: 'bm25_retrieval-0883a41b' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/85bedb6e-6ef2-4c08-9a48-4984b66a036c/compare?selectedSessions=864c5623-d304-4aba-8460-dd006cf7780d




...

0it [00:00, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the main factors contributing to the ongoing confusion among students regarding loan servicing after the end of the federal student loan forbearance period?  ', '2. How has the expiration of the federal student loan forbearance impacted the clarity and understanding of loan repayment processes for borrowers?  ', '3. In what ways has the conclusion of the federal student loan forbearance program led to challenges or misunderstandings in student loan servicing for borrowers?']
INFO:httpx:HTTP Reques

View the evaluation results for experiment: 'semantic_retrieval-0d1b701a' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/85bedb6e-6ef2-4c08-9a48-4984b66a036c/compare?selectedSessions=5a36a286-421c-4f25-9c38-4b82fd511b9e




0it [00:00, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https

In [152]:
langsmith_evaluation(parent_document_retriever, dataset_name)

INFO:__main__:Starting LangSmith evaluation for chain: ParentDocumentRetriever
INFO:__main__:Using dataset: Loan Synthetic Data - Advanced Retrieval


View the evaluation results for experiment: 'parent_document_retrieval-421f6365' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/85bedb6e-6ef2-4c08-9a48-4984b66a036c/compare?selectedSessions=998ea98c-2222-4ed0-995c-a280d982877b




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: argument 'text': 'dict' object cannot be converted to 'PyString'
Traceback (most recent call last):
  File "/Users/terellbrown/terellcodes/AIE7/09_Advanced_Retrieval/.venv/lib/python3.13/site-packages/langsmith/evaluation/_runner.py", line 1907, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
    ~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/AIE7/09_Advanced_Retrieval/.venv/lib/python3.13/site-packages/langchain_core/retrievers.py", line 259, in invoke
    result = self._get_relevant_documents(
        input, run_manager=run_manager, **kwargs_
    )
  File "/Users/terellbrown/terellcodes/AIE7/09_Advanced_Retrieval/.venv/lib/python3.13/site-packages/langchain/retrievers/multi_vector.py", line 78, in _get_relevant_documents
    sub_docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
  File "/Users/terellbrown/terellcodes/AIE7/09_Advanced_Retrieval/.venv/lib/

In [169]:
metrics = {}
for chain in retrievers:
  ragas_metrics = ragas_evaluation(chain, dataset)
  metrics[chain.name] = ragas_metrics

Starting RAGAS evaluation for chain: RunnableSequence
Processing 11 samples
Processing sample 1/11
eval_sample: user_input='why my payment go up so much after federal student loan COVID-19 forbearance program end?' retrieved_contexts=None reference_contexts=["The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers."] response=None multi_responses=None reference='The federal student loan COVID-19 forbearance pro

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The significant increase in your federal student loan payments after the COVID-19 forbearance program ended is likely due to the loans not being re-amortized or adjusted once the forbearance concluded. During forbearance, payments are typically paused or reduced, and interest may accrue depending on the specific circumstances. When the forbearance ended, the repayment terms may not have been automatically recalculated to reflect the temporary pause, resulting in larger monthly payments.

Some complaints in the context indicate that re-amortization or adjustment efforts were delayed or did not occur promptly, causing payments to nearly double or increase substantially. For example, one borrower reported that their payment nearly doubled from $180 to $360 because the loans were not re-amortized in a timely manner after the forbearance ended. Additionally, in some cases, interest may have continued to accrue during forbearance, further increasing the total amount owed and leadin

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Aidvantage assigned you an incorrect monthly payment amount on your Income-Driven Repayment (IDR) plan despite your income qualifying for a lower payment because their system appears to have calculated or billed an amount that is not consistent with your documented income and the applicable repayment plan. Despite your timely recertification and submission of the correct income information, they continued to bill an amount that was not legally or mathematically possible based on your income, and they did not process your application promptly, which may have contributed to this discrepancy.

In response to your inquiries, Aidvantage placed you in a two-month administrative forbearance and then continued to bill the same incorrect amount without processing your income-driven plan application or requesting further documentation, even though you repeatedly emphasized that your income qualifies for a lower payment. When you pressed for clarification, they claimed that your applica

...

RAGAS evaluation complete
Results: {'context_recall': 0.8203, 'faithfulness': 0.7286, 'factual_correctness(mode=f1)': 0.4445, 'answer_relevancy': 0.5861, 'context_entity_recall': 0.3961, 'noise_sensitivity(mode=relevant)': 0.1778}
Starting RAGAS evaluation for chain: RunnableSequence
Processing 11 samples
Processing sample 1/11
eval_sample: user_input='why my payment go up so much after federal student loan COVID-19 forbearance program end?' retrieved_contexts=None reference_contexts=["The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The increase in your federal student loan payments after the COVID-19 forbearance program ended is likely due to the loans being re-amortized based on the original loan terms. During the forbearance period, payments were temporarily paused or reduced, and interest may not have accrued or was accrued differently depending on the type of forbearance. 

Once the forbearance ended, the loans needed to be recalculated (re-amortized) to include any outstanding interest and to set new monthly payments based on the current loan balance and original repayment schedule. If this re-amortization was delayed or not handled promptly, your payments could have increased significantly—sometimes nearly doubling—since the loan was recalculated to cover the full amount owed over the remaining loan term.

Additionally, some issues noted in complaints include loans going into interest-only payments or balances increasing despite payments, which can also contribute to higher monthly payments after 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/rerank "HTTP/1.1 200 OK"


KeyboardInterrupt: 

In [172]:
multi_query_retrieval_ragas_metrics = ragas_evaluation(multi_query_retrieval_chain, dataset)


Starting RAGAS evaluation for chain: RunnableSequence
Processing 11 samples
Processing sample 1/11
eval_sample: user_input='why my payment go up so much after federal student loan COVID-19 forbearance program end?' retrieved_contexts=None reference_contexts=["The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers."] response=None multi_responses=None reference='The federal student loan COVID-19 forbearance pro

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the reasons behind the significant increase in my federal student loan payments after the COVID-19 forbearance period ended?', '2. How does the end of the COVID-19 forbearance program impact the repayment amount for federal student loans?', '3. Why did my federal student loan payments rise substantially once the COVID-19 student loan pause concluded?']
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The increase in your federal student loan payments after the COVID-19 forbearance program ended is likely due to a combination of factors related to how the end of the forbearance was handled and how your loan was re-amortized. Based on the information from the complaints, here are some common reasons:

1. **Delayed Re-Amortization**: When the forbearance or pause ended, some borrowers' loans were not immediately re-amortized to reflect the current repayment terms, leading to higher and sometimes nearly double payments. For example, one complaint mentioned payments rising from around $180 to $360 because re-amortization was not processed promptly.

2. **Lack of Notification or Communication**: Borrowers reported not being properly notified about the end of forbearance or the new payment amounts. This lack of communication can result in unanticipated payment increases.

3. **Interest Capitalization**: Several complaints indicate that interest continued to accrue during the for

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['What are the possible reasons Aidvantage might have set an incorrect monthly payment for my Income-Driven Repayment plan despite qualifying for a lower amount, and how have they addressed my concerns so far?  ', 'Can you explain why Aidvantage might have calculated an inaccurate monthly payment for my income-qualifying repayment plan, and what steps they have taken after I raised this issue?  ', 'What factors could have led to Aidvantage assigning the wrong monthly payment amount on my Income-Driven Repayment plan despite my income qualification, and how has the company responded to my questions or complaints?']
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embe

Response: Aidvantage assigned you an incorrect monthly payment amount on your Income-Driven Repayment (IDR) plan despite your income qualifying for a lower payment because there appears to be a systemic error or miscalculation on their part. Several complaints indicate that Aidvantage has repeatedly provided inaccurate payment information that does not align with borrowers' documented incomes. In your case, despite submitting your recertification on time and providing correct income information, they assigned you a payment amount that is not mathematically or legally consistent with your income level.

In response to your inquiries, Aidvantage has generally responded by closing the complaints with explanations, often stating that their system has not yet processed your application or that there are delays and system errors beyond your control. They have not requested additional documentation from you nor provided a clear, accurate recalculation or explanation of how the incorrect amoun

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What factors contributed to the widespread confusion among students regarding loan servicing after the end of the federal student loan forbearance program?', '2. How did the conclusion of the federal student loan forbearance lead to challenges and misunderstandings in loan servicing for borrowers?', '3. What are the main reasons behind the recent increases in borrower confusion and issues with student loan servicing following the expiration of the forbearance period?']
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The confusion surrounding student loan servicing after the federal student loan forbearance program ended stems from several factors highlighted in the complaints:

1. **Inconsistent and Inaccurate Communication:** Many borrowers report not receiving proper notifications about changes in their loan status, such as the end of forbearance, loan transfers, or changes in repayment plans. For example, borrowers mentioned not being notified when their loans were transferred to new servicers or when forbearances concluded, leading to uncertainty.

2. **Loan Transfers and Multiple Servicers:** Over the years, loans have been sold or transferred between various servicers, often without clear or timely communication. This causes confusion about who manages the loan and how to access accurate information.

3. **Discrepancies in Loan Information:** Complaints reveal issues such as unrecognized increases in loan balances during periods of supposed interest-free forbearance, incorrect repo

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:

RAGAS evaluation complete
Results: {'context_recall': 1.0000, 'faithfulness': 0.8868, 'factual_correctness(mode=f1)': 0.4745, 'answer_relevancy': 0.6872, 'context_entity_recall': 0.3492, 'noise_sensitivity(mode=relevant)': 0.3333}


{'context_recall': 1.0000, 'faithfulness': 0.8868, 'factual_correctness(mode=f1)': 0.4745, 'answer_relevancy': 0.6872, 'context_entity_recall': 0.3492, 'noise_sensitivity(mode=relevant)': 0.3333}

In [ ]:
# Contextual Compression (Rerank) Evaluation
contextual_compression_ragas_metrics = ragas_evaluation(contextual_compression_retrieval_chain, dataset)
contextual_compression_ragas_metrics


Starting RAGAS evaluation for chain: RunnableSequence
Processing 11 samples
Processing sample 1/11
eval_sample: user_input='why my payment go up so much after federal student loan COVID-19 forbearance program end?' retrieved_contexts=None reference_contexts=["The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers."] response=None multi_responses=None reference='The federal student loan COVID-19 forbearance pro

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The increase in your federal student loan payments after the end of the COVID-19 forbearance program can be due to several factors:

1. **Re-amortization of your loans:** During the forbearance period, interest usually did not accrue or was handled differently, and payments may have been temporarily paused or reduced. Once the forbearance ended, loans needed to be re-amortized, meaning the monthly payments were recalculated based on the current balance, interest rate, and remaining term. If the original repayment plan was extended or if interest accrued during forbearance, your new payments could be substantially higher.

2. **Interest Accumulation During Forbearance:** Some accounts continued to accrue interest even during the period of forbearance, increasing your total balance. When payments resume, you might be paying off this accumulated interest in addition to the principal, leading to higher monthly payments.

3. **Loan Servicer Actions:** According to complaints, ther

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Based on the complaints provided, Aidvantage assigned some borrowers an incorrect monthly payment amount on their Income-Driven Repayment (IDR) plans despite their income qualifying them for a lower payment. The main reasons for this seem to include:

1. **Application Processing Delays:** Although borrowers submitted recertification documents on time and their applications were marked as approved or in processing, Aidvantage sometimes delayed applying the correct calculation. In some cases, the application was marked as "not processed yet," yet bills were issued based on recalculations, which is contradictory.

2. **Lack of Communication and Clarification:** Aidvantage often did not request additional documentation or communicate clearly about the processing status. This led to situations where borrowers continued to be billed at incorrect amounts despite their applications being approved.

3. **Administrative Errors and Delays:** There are instances where the system or staff

...

Response: There is so much confusion surrounding student loan servicing after the end of the federal student loan forbearance program due to several factors. First, there have been issues with communication and transparency from loan servicers, such as Nelnet and MOHELA, leading borrowers to be unaware of changes to their loan status, repayment obligations, or account transfers. Some borrowers received conflicting or incomplete information—such as claims of extended forbearance periods or being told they are still in forbearance until far in the future, often without any written confirmation.

Second, the resumption of payments and loan transfers have not been well coordinated or clearly communicated, resulting in problems like misreported account statuses on credit reports, unexpected repayment amounts, or unnotified loan transfers to different servicers, with borrowers having to navigate multiple websites to find their loan information.

Third, many borrowers experienced difficulties

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:

KeyboardInterrupt: 

ERROR:ragas.executor:Exception raised in Job[19]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[2]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[8]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[7]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[26]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[28]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[25]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[23]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[17]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[11]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[12]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[30]: AssertionError(set LLM before use)
ERROR:ragas.executor:Exception raised in Job[31]: AssertionError(LLM is not

In [ ]:
# Parent Document Retrieval Evaluation
parent_document_ragas_metrics = ragas_evaluation(parent_document_retrieval_chain, dataset)
parent_document_ragas_metrics


Starting RAGAS evaluation for chain: RunnableSequence
Processing 11 samples
Processing sample 1/11
eval_sample: user_input='why my payment go up so much after federal student loan COVID-19 forbearance program end?' retrieved_contexts=None reference_contexts=["The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers."] response=None multi_responses=None reference='The federal student loan COVID-19 forbearance pro

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: After the end of the federal student loan COVID-19 forbearance program, your payments may have increased significantly because the loans were not automatically re-amortized or recalculated to reflect the current repayment terms. During forbearance, payments were often suspended or reduced, but once the forbearance ends, the original loan repayment terms typically resume unless the lender or servicer re-amortizes the loan to spread the remaining balance over the remaining term. 

In your case, it appears that your loan servicer, Nelnet, did not re-amortize your loan immediately after the forbearance ended, resulting in a much higher monthly payment. This situation led to your new payment nearly doubling, from around $180 to $360. The re-amortization process should ideally have occurred automatically to avoid sudden increases in payment amount, but delays or oversights can happen, causing the payment to go up substantially.

If you believe the increase was due to an oversight o

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Aidvantage assigned you an incorrect monthly payment amount on your Income-Driven Repayment (IDR) plan despite your income qualifying for a lower payment because it appears there was a systemic error or miscalculation on their end. Several complaints indicate that Aidvantage has repeatedly billed borrowers an amount that does not align with their documented income and eligibility for a lower payment under the IDR guidelines. In some cases, borrowers report that Aidvantage has not processed recertification properly or has used incorrect data to determine the payment amount, leading to overbilling.

In response to inquiries, Aidvantage has generally responded by stating that their processing of the application is pending or has not been completed yet, even though they have issued bills based on recalculations. They have also placed some borrowers into administrative forbearance while attempting to resolve these errors. However, they have not provided clear explanations, request

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Based on the information provided, federal student loan payments typically need to start after you graduate, leave school, or drop below half-time enrollment. However, the exact timing can vary depending on your loan type and when your loan servicer or the Department of Education notifies you to begin repayment. 

In your specific case, some individuals received notifications asking them to start paying back their loans after many years, even if their loans had been paid in full or they were unaware of repayment requirements. For example, one complaint mentions receiving an email saying it was time to start paying again years after the loan was paid off. 

Since I don’t have your specific loan details, the safest answer is:  
**You should start paying your federal student loans once you receive official notification from your loan servicer or the Department of Education indicating that repayment is due.**  

If you haven't received such notification, it may be best to contact

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The confusion stemming from conflicting information about loan servicers and lack of proper notification about servicer changes significantly harms the process and accuracy of Income-Driven Repayment (IDR) recertification. When borrowers do not receive clear, timely disclosures about changes in their servicers—such as notices from studentaid.gov versus information from the servicers themselves—they may be unaware of where their loans are hosted or who is handling their repayment plans. This lack of transparency can lead to missed communications, delayed processing of recertification applications, and errors in account records.

Such issues are illustrated in complaints involving both Nelnet and Aidvantage. For instance, borrowers report not being properly informed about servicer changes or experiencing difficulty in obtaining accurate information regarding their loan status and repayment obligations. When borrowers are uncertain about their loan servicer or do not receive not

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
# Ensemble Retrieval Evaluation
ensemble_retrieval_ragas_metrics = ragas_evaluation(ensemble_retrieval_chain, dataset)
ensemble_retrieval_ragas_metrics


In [ ]:
# Semantic Retrieval Evaluation
semantic_retrieval_ragas_metrics = ragas_evaluation(semantic_retrieval_chain, dataset)
semantic_retrieval_ragas_metrics


In [ ]:
metrics_consolidated = {
    "naive_retrieval_chain" : metrics["naive_retrieval_chain"],
    "multi_query_retrieval_chain" : multi_query_retrieval_ragas_metrics,
    "bm25_retrieval_chain" : metrics["bm25_retrieval_chain"],
    "contextual_compression_retrieval_chain" : contextual_compression_ragas_metrics,
    "parent_document_retrieval_chain" : parent_document_ragas_metrics,
    "ensemble_retrieval_chain" : ensemble_retrieval_ragas_metrics,
    "semantic_retrieval_chain" : semantic_retrieval_ragas_metrics
}

In [ ]:
metrics_consolidated

{'naive_retrieval': {'context_recall': 0.9773, 'faithfulness': 0.8540, 'factual_correctness(mode=f1)': 0.5300, 'answer_relevancy': 0.8514, 'context_entity_recall': 0.4308, 'noise_sensitivity(mode=relevant)': 0.2835},
 'bm25_retrieval': {'context_recall': 0.8203, 'faithfulness': 0.7286, 'factual_correctness(mode=f1)': 0.4445, 'answer_relevancy': 0.5861, 'context_entity_recall': 0.3961, 'noise_sensitivity(mode=relevant)': 0.1778}}